In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

from pathlib import Path

import keras_tuner as kt

from importlib import reload
import hyper_model_creator
import keras_model_creator
import sentinel_utils

In [2]:
trials_dir = Path('trials', 'hyperband_resnet')
trial_metric = 'val_weightedf2score'

In [3]:
callbacks = [
    tf.keras.callbacks.TensorBoard(trials_dir.joinpath('board')),
    tf.keras.callbacks.EarlyStopping(
        monitor=trial_metric, patience=20, mode='max'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor=trial_metric, factor=0.5, patience=8, min_lr=1e-6,
    ),
]

In [ ]:
reload(hyper_model_creator)

hypermodel = hyper_model_creator.BuildHyperModel()

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=trials_dir.parent,
    project_name=trials_dir.name,
    hyperband_iterations=3,
    max_epochs=100,
    overwrite=False,
)
tuner.search(
    verbose=1, 
    callbacks=callbacks
)

Reloading Tuner from trials/hyperband_resnet/tuner0.json

Search: Running Trial #467

Value             |Best Value So Far |Hyperparameter
True              |True              |class_weight
2017_2018_2019    |2017_2018_2019    |training_years
l1l2              |l1l2              |kernel_regularizer
0.1               |0.1               |spatial_dropout
relu              |relu              |activation
4                 |4                 |pool_size
0.5               |0.5               |dropout
True              |True              |bias_initializer
0.0001            |0.0001            |learning_rate
binary_crossent...|binary_crossent...|loss_function
100               |34                |tuner/epochs
34                |12                |tuner/initial_epoch
2                 |2                 |tuner/bracket
2                 |1                 |tuner/round
0461              |0456              |tuner/trial_id



/home/pj/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 102 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 35/100


I0000 00:00:1724319235.143094   25490 service.cc:145] XLA service 0x7f2298013e40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724319235.143145   25490 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
I0000 00:00:1724319246.106887   25490 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3550/3550 ━━━━━━━━━━━━━━━━━━━━ 243s 64ms/step - accuracy: 0.5531 - auc: 0.9434 - loss: 0.2541 - prc: 0.8526 - precision: 0.7962 - recall: 0.7179 - weightedf1score: 0.7389 - weightedf2score: 0.7251 - val_accuracy: 0.5166 - val_auc: 0.9210 - val_loss: 0.2986 - val_prc: 0.8052 - val_precision: 0.7561 - val_recall: 0.6830 - val_weightedf1score: 0.6978 - val_weightedf2score: 0.6880 - learning_rate: 1.0000e-04
Epoch 36/100
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 233s 66ms/step - accuracy: 0.5545 - auc: 0.9437 - loss: 0.2534 - prc: 0.8534 - precision: 0.7984 - recall: 0.7183 - weightedf1score: 0.7394 - weightedf2score: 0.7255 - val_accuracy: 0.5107 - val_auc: 0.9199 - val_loss: 0.3019 - val_prc: 0.8025 - val_precision: 0.7548 - val_recall: 0.6769 - val_weightedf1score: 0.6904 - val_weightedf2score: 0.6811 - learning_rate: 1.0000e-04
Epoch 37/100
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 233s 65ms/step - accuracy: 0.5576 - auc: 0.9449 - loss: 0.2507 - prc: 0.8565 - precision: 0.8001 - recall: 0.7212 - weightedf1s

In [ ]:
# best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

# best_hp.values['units'] = 128
# best_hp.values['dropout'] = 0.3

# model = tuner.hypermodel.build(best_hp)
# model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

In [ ]:
# tuner.results_summary(num_trials=2)

In [ ]:
# %load_ext tensorboard
# tb_path = str(trials_dir.joinpath('board'))
# %tensorboard --logdir "$tb_path"